In [4]:
import numpy as np

import nibabel as nib

nii_file = '/home/abigail/mphys/7a7/CT_Whole_CNS.nii' 
img = nib.load(nii_file)
data = img.get_fdata()

crop_in_x_start = 100
crop_in_x_end = -100
crop_in_y_start = 100
crop_in_y_end = -100
crop_in_z = -160 #negative because it only wants the last elements

cropped_data = data[crop_in_x_start:crop_in_x_end, crop_in_y_start:crop_in_y_end:, crop_in_z:]

cropped_img = nib.Nifti1Image(cropped_data, affine=img.affine, header=img.header)

nib.save(cropped_img, '/home/abigail/mphys/7a7/CT_Whole_CNS_cropped.nii')

print("Cropped shape:", cropped_data.shape)

Cropped shape: (312, 312, 160)


In [5]:
%matplotlib inline

import matplotlib.pyplot as plt
import nibabel as nib
import numpy as np
import ipywidgets as widgets
from IPython.display import display, clear_output

img = nib.load("/home/abigail/mphys/7a7/CT_Whole_CNS_cropped.nii")
X = img.get_fdata()
X = (X - np.min(X)) / (np.max(X) - np.min(X))

affine = img.affine
voxel_sizes = np.sqrt((affine[:3, :3] ** 2).sum(axis=0))
dx, dy, dz = voxel_sizes

def show_slice(slice_index):
    plt.figure(figsize=(6, 6))
    slice_img = X[:, slice_index, :].T

    plt.imshow(slice_img, cmap='gray', origin='lower', 
               aspect=dz / dx)
    plt.title(f"Coronal Slice {slice_index}/{X.shape[1]-1}")
    plt.axis("off")
    clear_output(wait=True)
    display(plt.gcf())
    plt.close()

slider = widgets.IntSlider(
    value=X.shape[1] // 2,
    min=0,
    max=X.shape[1] - 1,
    step=1,
    description='Slice:',
    continuous_update=True,
    layout=widgets.Layout(width='80%')
)

widgets.interact(show_slice, slice_index=slider)


interactive(children=(IntSlider(value=156, description='Slice:', layout=Layout(width='80%'), max=311), Output(…

<function __main__.show_slice(slice_index)>

In [1]:
from totalsegmentator.python_api import totalsegmentator

def main():
    input_path=r"Z:\FacialDeformation_MPhys\paired_data\nina_abby\UIDQQ0Q00Q910\CT_WHOLE_CNS_cropped.nii"
    output_path= r"Z:\FacialDeformation_MPhys\paired_data\nina_abby"

    totalsegmentator(input_path,output_path,task = 'craniofacial_structures')
    print('\nDone!')

if __name__ == "__main__":
 main()

ModuleNotFoundError: No module named 'totalsegmentator'

In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt
import nibabel as nib
import numpy as np
import ipywidgets as widgets
from IPython.display import display, clear_output

ct_img = nib.load("/home/abigail/mphys/1Zr/CT_WHOLE_CNS_cropped.nii")
X = ct_img.get_fdata()
X = (X - np.min(X)) / (np.max(X) - np.min(X))  # Normalize to [0, 1] (binary)

seg_paths = {
    "head": "/home/abigail/mphys/1Zr/head.nii.gz",
    "mandible": "/home/abigail/mphys/1Zr/mandible.nii.gz",
    "sinus_frontal": "/home/abigail/mphys/1Zr/sinus_frontal.nii.gz",
    "sinus_maxillary": "/home/abigail/mphys/1Zr/sinus_maxillary.nii.gz",
    "skull": "/home/abigail/mphys/1Zr/skull.nii.gz",
    "teeth_upper": "/home/abigail/mphys/1Zr/teeth_upper.nii.gz",
    "teeth_lower": "/home/abigail/mphys/1Zr/teeth_lower.nii.gz",
}

segments = {name: nib.load(path).get_fdata() for name, path in seg_paths.items()}
spacing = ct_img.header.get_zooms()

overlay_colors = {
    "head": (0.2, 0.4, 1.0, 0.25),          # blueish
    "mandible": (1.0, 0.6, 0.0, 0.35),      # orange
    "sinus_frontal": (0.2, 1.0, 0.2, 0.35), # green
    "sinus_maxillary": (0.6, 0.2, 1.0, 0.35), # purple
    "skull": (0.9, 0.9, 0.8, 0.25),         # bone
    "teeth_upper": (1.0, 0.2, 0.2, 0.35),   # red
    "teeth_lower": (1.0, 0.6, 0.4, 0.35),   # light red/orange
}

def show_slice(slice_index, plane, alpha=0.35, patient_id="UIDQQ0x9x011Zr"):
    plt.figure(figsize=(6, 6))

    if plane == 'Axial':
        img_slice = np.rot90(X[:, :, slice_index])
        aspect = spacing[1] / spacing[0]
        seg_slices = {n: np.rot90(S[:, :, slice_index]) for n, S in segments.items()}

    elif plane == 'Coronal':
        img_slice = np.flipud(np.rot90(X[:, slice_index, :]))
        aspect = spacing[2] / spacing[0]
        seg_slices = {n: np.flipud(np.rot90(S[:, slice_index, :])) for n, S in segments.items()}

    elif plane == 'Sagittal':
        img_slice = np.flipud(np.rot90(X[slice_index, :, :]))
        aspect = spacing[2] / spacing[1]
        seg_slices = {n: np.flipud(np.rot90(S[slice_index, :, :])) for n, S in segments.items()}

    plt.imshow(img_slice, cmap='gray', origin='lower', aspect=aspect)

    for name, seg_slice in seg_slices.items():
        mask = seg_slice > 0.5
        if np.any(mask):
            rgba = list(overlay_colors[name])
            rgba[3] = alpha  
            overlay = np.zeros((*mask.shape, 4))
            overlay[mask] = rgba
            plt.imshow(overlay, origin='lower', aspect=aspect)

    plt.title(f"{plane} slice {slice_index}")
    plt.axis("off")


    legend_handles = [
        plt.Line2D([0], [0], color=overlay_colors[n][:3], lw=4, label=n)
        for n in overlay_colors
    ]
    leg=plt.legend(handles=legend_handles, loc='upper right', fontsize=8, frameon=False, labelcolor='white', title="Segments")
    leg.get_title().set_fontsize(10)
    leg.get_title().set_color('white')
    clear_output(wait=True)
    display(plt.gcf())
    plt.close()


plane_dropdown = widgets.Dropdown(
    options=['Axial', 'Coronal', 'Sagittal'],
    value='Axial',
    description='Plane:'
)

slice_slider = widgets.IntSlider(
    value=X.shape[2] // 2,
    min=0,
    max=X.shape[2] - 1,
    step=1,
    description='Slice:',
    continuous_update=True,
    layout=widgets.Layout(width='80%')
)

alpha_slider = widgets.FloatSlider(
    value=0.35,
    min=0.0,
    max=1.0,
    step=0.05,
    description='Opacity:',
    continuous_update=True,
    layout=widgets.Layout(width='60%')
)


def update_slider_range(*args):
    plane = plane_dropdown.value
    if plane == 'Axial':
        slice_slider.max = X.shape[2] - 1
        slice_slider.value = X.shape[2] // 2
    elif plane == 'Coronal':
        slice_slider.max = X.shape[1] - 1
        slice_slider.value = X.shape[1] // 2
    elif plane == 'Sagittal':
        slice_slider.max = X.shape[0] - 1
        slice_slider.value = X.shape[0] // 2


plane_dropdown.observe(update_slider_range, names='value')
update_slider_range()

widgets.interact(show_slice, slice_index=slice_slider, plane=plane_dropdown, alpha=alpha_slider)


interactive(children=(IntSlider(value=80, description='Slice:', layout=Layout(width='80%'), max=159), Dropdown…

<function __main__.show_slice(slice_index, plane, alpha=0.35, patient_id='UIDQQ0x9x011Zr')>